# Tutoriaali 11

Ryhdymme kurssin loppua kohden käsittelemään yksittäisten kuvien sijaan koko
kuva-aineistoa. Tällä kerralla opettelemme organisoimaan kokeilumme skripteiksi
ja kokeilemme tilastollisen datan kokoamista kaikista valitsemistamme kuvista.
Ryhdymme myös vähitellen harjoittelemaan perusmenetelmiä datan analysoimiseksi
ja visualisoimiseksi.

Tässä vaiheessa kaikki ovat jo tehneet monenlaisia kokeiluja yksittäisillä
kuvilla ja löytäneet joitakin lupaavia tapoja esikäsitellä kuvia sopivampaan
muotoon ongelman ratkaisemiseksi. Seuraava vaihe on valita joukko operaatioita,
suorittaa ne kaikille kuville ja tallentaa lopputulos kuvatiedostoiksi.
Riippuen valituista operaatioista saattaa olla järkevää tallentaa useita
välivaiheita.

Seuraavaksi kaikki tuloskuvat pitäisi käydä läpi ja arvioida, miten hyvin
valitut operaatiot toimivat koko kuvajoukolle. Saattaa olla tarpeen säätää
joitakin parametreja tai lisätä välivaiheita, jotta lopputulos olisi
mahdollisimman hyvä mahdollisimman monelle kuvalle. On hyvin todennäköistä, että
kuvajoukossa on joitakin kuvia, joille valitut operaatiot eivät toimi. Tässä
tilanteessa ei kannata käyttää liikaa aikaa säätämiseen, vaan tyytyä
epätäydelliseen tulokseen tai poistaa ongelmalliset kuvat. Voi olla tarpeen
pohtia, miten ilmenneet ongelmat ratkaistaisiin valmiissa sovelluksessa.

## Esimerkki

Tutkitaan yksinkertaisia geometrisia muotoja, kolmioita, neliöitä ja ympyröitä,
jotka ovat vääristyneet eri tavoin. Lopullisessa sovelluksessa olisi tarkoitus
selvitä myös valaistuksen vaihteluista ja varjoista, mutta tässä vaiheessa
valaistus on yksinkertaisuuden vuoksi tasainen. Kohinaa on jonkin verran.

Datajoukon kuvat voi ladata [täältä](http://users.jyu.fi/~amjayee/TIES411/shapes_v1.zip). Seuraavassa muutama 
esimerkki kuvista. Jotta tämä työkirja toimisi, kuvapaketti pitää purkaa samaan kansioon työkirjan kanssa.

![Esimerkki datajoukon kuvista](./shapes.png)

Tehdään skripti, joka lukee kaikki kuvat vuoron perään, suorittaa niille joukon
operaatioita, ja tallentaa lopputuloksen kuvana uuteen tiedostoon tarkastelua
varten. Tässä kohtaa on syytä huomata, että kuvia on parasta käsitellä yksi
kerrallaan vuoron perään. Muisti loppuu hyvin pian nykyaikaisistakin
tietokoneista, jos muistiin avataan samanaikaisesti satoja suuria kuvia. Tässä
datajoukossa on 300 kpl noin 40 kilotavun kokoisia kuvia, joten niitä vielä
pystyisi käsittelemään muistissa samanaikaisestikin, mutta tyypillisen
datajoukon kuvat ovat huomattavasti suurempia.

Ladataan taas aluksi tarvittavat kirjastot. Uutena tuttavuutena ryhdymme
kokeilemaan pandas-kirjastoa datan käsittelyyn ja visualisointiin. Käytämme
myös python `csv`-moduulia yksinkertaisena esimerkkinä datan lukemisesta.

In [ ]:
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

%matplotlib inline

## CSV-tiedostojen käsittely

On hyvä ajatus säilyttää dataa jossakin helposti luettavassa ja käsiteltävässä
muodossa. Esimerkissämme käytämme CSV-tiedostoja, sillä

* ne ovat helposti ihmisen luettavissa ja muokattavissa myös tekstieditorissa,
* ne saa luettua sellaisenaan esimerkiksi Exceliin ja moniin muihinkin
  työkaluihin jatkokäsittelyä ja visualisointeja varten, ja
* tarvittaessa on helppo koodata itsekin työkaluja niiden käsittelyyn.

Lähdemme liikkeelle siitä, että kuvailemme alkuperäisen datajoukkomme
CSV-tiedoston riveinä. Kirjaamme tiedostoon kunkin kuvan polun ja tiedostonimen
suhteessa tämän CSV-tiedoston sijaintiin, sekä kuvan esittämän kohteen luokan
kokonaislukuina $1$, $2$, $3$. Sovelluksesta riippuen kuviin voi liittää mitä
tahansa muutakin ennakkoon tiedossa olevaa tietoa, ja esimerkiksi loppuun voi
jättää sarakkeen mahdollisia kommentteja varten.

Saamme helposti muodostettua pohjan tiedostoa varten keräämällä kuvatiedostojen
nimet jonkin kansion alikansioista vaikkapa komennolla

```{.bash}
ls -Rpx1 shapes > shapes.csv
```

Kirjoittamalla shell-skriptin saa liitettyä tiedostonimeen myös polun, mutta
tämä kertaluontoinen työ on kohtuullisella vaivalla tehtävissä myös käsin
tekstieditorissa.

Tällaista datajoukon kuvailevaa tiedostoa kutsutaan joskus *manifestiksi*, joka
on siis eräänlainen pakkausluettelo. Se auttaa systematisoimaan datan
käsittelyä, mikä puolestaan on avuksi erilaisten kokeilujen toistamisessa
tarpeen mukaan. Jos haluaa vaikkapa jättää osan kuvista pois kokeiluista
väliaikaisesti tai kokonaan, tämän voi tehdä esimerkiksi lisäämällä manifestiin
ylimääräisen sarakkeen, jonka arvon perusteella kuva jätetään käsittelemättä.
Kommenttisarake auttaa muistamaan, mikä mahdollisesti oli syynä tiedoston pois
jättämiseen, ja jos myöhemmin haluaa kokeilla jotakin uutta, kuva on edelleen
olemassa ja sen saa helposti palautettua käsittelyyn. Jos taas kuvia tuhoaa ja
lisää summittaisesti, eikä erilaisia kokeiluja dokumentoi millään tavalla, on
todella hankalaa palata myöhemmin takaisin johonkin aiempaan tilanteeseen.

Käytetään CSV-tiedoston lukemiseen pythonin `csv`-modulia. Sen avulla tiedoston
rivit saa luettua kätevästi `dict`-olioiksi, jos csv-tiedoston ensimmäisellä
rivillä on määritelty sarakkeiden nimet. Jos nimiä ei ole tiedostossa, ne saa
myös kerrottua konstruktorin `fieldnames`-parametrilla. Rivien lukemisen
yhteydessä on hyvä ajatus muuntaa sarakkeiden sisältö oikean tyyppiseksi.
Esimerkissämme haluamme varmistaa, että luokkasarakkeen arvot ovat
kokonaislukuja.  Konstruktorin parametreilla voi myös määritellä esimerkiksi
sarakkeiden ja rivien erottimet, jos tiedoston formaatti on jokin muu kuin
perinteinen.

In [ ]:
def read_data(filename):
  result = []
  with open(filename, 'r') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=",", skipinitialspace=True, quotechar="'", lineterminator="\n")
    for row in reader:
      row["class"] = np.int32(row["class"])
      result.append(row)
  return result

shapes = read_data('shapes.csv')
print(shapes[0])

## Datan käsittely

Tehdään kuville yksinkertainen gaussinen suodatus kohinan poistamiseksi sekä
kynnystys käyttäen kiinteää kynnysarvoa. Kerätään samalla kuvista esimerkin
vuoksi kohteeseen kuuluvien pikselien kokonaismäärä, jota kutsumme 'massaksi'.
Kirjoitetaan tuloksena syntyvä kuva toiseen tiedostoon, jotta voimme 
tarvittaessa tarkistaa, millaiseen lopputulokseen kerätty data perustuu.
Kirjoitetaan käsittelyoperaatioista sellainen funktio, jota voimme käyttää 
`map`-operaatiolla koko datajoukkoon. Se ottaa siis parametrina csv-tiedostosta
luetun `dict`-olion ja paluttaa muokatun olion.

In [ ]:
def process(shape):
  filename = shape["file"]
  img = cv.GaussianBlur(np.float32(cv.imread(filename, cv.IMREAD_GRAYSCALE)), (5,5), 1)
  t = np.float32(img > 100)
  m = np.sum(t)
  shape["mass"] = m
  cv.imwrite(filename.replace('images', 'results'), t*255)
  return shape

output = list(map(process, shapes))
print(output[0])

Funktiosta palautuu takaisin siis vain `dict`-olio täydennettynä kerätyillä
tiedoilla. Ladattu kuva tuhoutuu aikanaan funktiosta poistumisen jälkeen, joten
se ei jää kuormittamaan muistia. Lopputuloksen tallentamiseksi
csv-tiedostoon käytämme edelleen `csv`-modulia. Tiedostoon kirjoitettaessa
sarakkeiden nimet täytyy luetella, jotta kirjoittaja tietää sarakkeiden oikean
järjestyksen.

In [ ]:
def write_data(filename, data):
  with open(filename, 'w') as resultfile:
    writer = csv.DictWriter(resultfile, fieldnames=["file","class","mass"], delimiter=",", quotechar="'", quoting=csv.QUOTE_MINIMAL, lineterminator="\n")
    writer.writeheader()
    writer.writerows(data)

write_data('results.csv', output)

Voimme nyt toteuttaa esimerkkikokeilumme yksinkertaisesti lukemalla datajoukon
tiedot manifestista, suorittamalla käsittelyfunktion koko datajoukolle
`map`-operaation avulla, ja kirjoittamalla lopputuloksen uuteen tiedostoon.
Näin saamme helposti dokumentoitua kokeilumme tekemällä kustakin oman skriptin
ja tallentamalla tulokset eri tiedostoihin ja kuvakansioihin.

Ryhdymme nyt kokeilemaan alussa käyttöön ottamaamme `pandas`-kirjastoa. Se käsittelee dataa `DataFrame`-olioina, joilla on monenlaisia hyödyllisiä ominaisuuksia. Saamme muodostettua `DataFrame`-olion suoraan samasta datasta, jonka kirjoitimme edellä csv-tiedostoon. Lukemisen onnistumisen saa varmistettua piirtämällä taulukosta muutaman ensimmäisen rivin `head()`-funktiolla.

In [ ]:
sd = pd.DataFrame(output)
sd.head()

`DataFrame` koostuu riveistä, joille voi muodostaa indeksejä, sekä nimetyistä sarakkeista vastaavasti kuin csv-tiedosto. Voimme myös esimerkiksi lukea csv-tiedoston suoraan `DataFrame`-olioksi. Samaan tapaan kuin pythonin `csv`-moduuli, myös `pandas` lukee sarakkeiden nimet otsikkoriviltä, ja ne voi tarvittaessa antaa käsin. Sarakkeiden tyypit voi myös määritellä.

In [ ]:
data = pd.read_csv("./results.csv", dtype={"file":"str", "class":np.int32, "mass":np.float32})
data.head()

Datasta saa helposti perustietoja ulos `describe()`-funktiolla.

In [ ]:
data.describe()

Taulukosta saa irrotettua yhden sarakkeen normaalilla hakasulkumerkinnällä. Valikoidusta ja suodatetusta datan osasta saa erilaisia koostetietoja, esimerkiksi `unique()`-funktiolla saamme haettua sarakkeessa `class` esiintyvät yksilölliset arvot. Tämä on hyödyllistä varsinkin tutustuttaessa uuteen datasettiin.

In [ ]:
data["class"].unique()

Taulukkoa saa suodatettua samaan tapaan kuin numpy-matriiseja käyttämällä indeksointioperaatioita. Voimme esimerkiksi erotella eri luokkiin kuuluvat rivit omiksi taulukoikseen.

In [ ]:
triangles = data[data["class"] == 1]
quads = data[data["class"] == 2]
circles = data[data["class"] == 3]
circles.head()

Yksittäisiä rivejä voi hakea indekseillä käyttäen `.loc()`-funktiota. Tässä on syytä huomata, että indeksi ei ole sama asia kuin tietyn taulukon rivinumero. Yllä näemme, että `circles`-taulukossa indeksin arvot alkavat edelleen $200$:sta. On tietysti hyödyllistä, että indeksit pysyvät samoina taulukkoa myllätessä, mutta joskus on hyvä saada haettua esille tietty rivi. Tämän saa tehtyä `.iloc()`-funktiolla.

In [ ]:
samples = [ triangles["file"].iloc[0]
          , quads["file"].iloc[0]
          , circles["file"].iloc[0] ]
timg = cv.imread(samples[0], cv.IMREAD_GRAYSCALE)
qimg = cv.imread(samples[1], cv.IMREAD_GRAYSCALE)
cimg = cv.imread(samples[2], cv.IMREAD_GRAYSCALE)

plt.figure(figsize=(15,5))
_ = plt.subplot(131),plt.xticks(()),plt.yticks(()),plt.imshow(timg, cmap="gist_gray"),plt.title('triangle')
_ = plt.subplot(132),plt.xticks(()),plt.yticks(()),plt.imshow(qimg, cmap="gist_gray"),plt.title('quad')
_ = plt.subplot(133),plt.xticks(()),plt.yticks(()),plt.imshow(cimg, cmap="gist_gray"),plt.title('circle')

Taulukon sarakkeista voi myös laskea erilaisia tunnuslukuja.

In [ ]:
m1 = triangles["mass"].sum() / triangles.size
m2 = quads["mass"].sum() / quads.size
m3 = circles["mass"].sum() / circles.size
s = max(m1,m2,m3)
print((m1/s,m2/s,m3/s))

Näemme, että eri objektien keskimääräinen massa on suunnilleen sama (mikä on tietysti tarkoituksellista). Luonnollisesti keskiarvon laskemista varten on myös valmis funktio.

In [ ]:
triangles["mass"].mean()

Taulukosta saa myös piirrettyä erilaisia kaavioita, voimme esimerkiksi tehdä kullekin luokalle erilliset histogrammit kohteiden massan jakaumasta. 

In [ ]:
_=data.hist(column="mass", by="class", sharex=True, sharey=True, range=(5500,8000), bins=30, layout=(1,3), figsize=(12,3))

Parametreilla saamme asetettua kaikkien histogrammien arvoalueet samoiksi, jolloin ne ovat keskenään verrattavissa. Näemme, että vaikka kaikkien luokkien massan keskiarvo on suunnilleen sama, arvojen jakauma on melko erilainen.

Käyttämällä komentoa `plt.figure()` on mahdollista piirtää useampi kaavio samaan koordinaatistoon. Voimme esimerkiksi piirtää arvioidun tiheysfunktiona massan jakaumille.

In [ ]:
plt.figure(figsize=(5,5))
_=triangles["mass"].plot.density()
_=quads["mass"].plot.density()
_=circles["mass"].plot.density()

Jatkossa tutustumme tarkemmin `pandas`-kirjastoon ja mitä kaikkea sillä voi tehdä. Netistä löytyy myös paljon erilaisia tutoriaaleja ja kursseja, joista voi oppia enemmän yksityiskohtia tästä suositusta ja tehokkaasta työkalusta.

## Tehtäviä

Tee python-skripti, joka lukee kaikki aineistosi kuvat, suorittaa niille
valitsemasi esikäsittelyoperaation ja tallentaa tulokset erilliseen kansioon.
Tarvittaessa voit tallentaa erikseen useita välivaiheita.

Käy läpi tulokset. Jos kuvia on paljon, esimerkiksi yli sata, riittää käydä läpi
jokin edustava osajoukko. Arvioi, miten hyvin valitsemasi operaatio toimii.
Pitäisikö jotakin parantaa? Kuinka ongelmia voisi mahdollisesti yrittää välttää
tai korjata lopullisessa sovelluksessa?

Mieti samalla jo alustavasti, millaisia numeerisia piirretietoja voisit laskea
kuvista ongelmasi ratkaisemista varten.